# Assigning Parameters for *pop*-terphenyl oligomers

This notebook is setup to assign parameters for a several lengths of oligomers for the *pop*-terphenyl system. This workflow is based off of the OpenFF example [here](https://github.com/openforcefield/openff-toolkit/tree/main/examples/using_smirnoff_in_amber_or_gromacs). This notebook relies on structures generated with `mbuild` from `/heteropolymer_simulations/examples/build_polymer`, so make sure the structure files from those examples are generated.

## Prepare an OpenFF and OpenMM Topologies

In [1]:
try:
    from openmm import app
except ImportError:
    from simtk.openmm import app

from openff.toolkit.topology import FrozenMolecule, Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.interchange.components.interchange import Interchange
import pdb
import os

In [2]:
mol_files = [
    "../../build_polymer/pop/pop_tetramer_mbuild.mol",
    "../../build_polymer/pop/pop_hexamer_mbuild.mol",
    "../../build_polymer/pop/pop_octamer_mbuild.mol",
    "../../build_polymer/pop/pop_decamer_mbuild.mol"
]

molecules = [Molecule.from_file(fn) for fn in mol_files]

In [3]:
openmm_topologies = []
openff_topologies = []
pdbfiles = []
for i in range(len(mol_files)):
    prefix = mol_files[i].split(".mol")[0]
    pdb_file = app.PDBFile(prefix + "_renum.pdb")
    omm_topology = pdb_file.topology
    off_topology = Topology.from_openmm(omm_topology, unique_molecules = [molecules[i]])
    openmm_topologies.append(omm_topology)
    openff_topologies.append(off_topology)
    pdbfiles.append(pdb_file)

## Assign partial charges using AM1-BCC

Here we assign partial charges uses AM1-BCC method. This method scales poorly for large systems, so for longer oligomers, like the octamer and decamer this may take several hours. For systems with more than 300 atoms a modified version of the OpenEye toolkit is needed to make sure AM1-BCC calculations can be run on systems larger than 300 atoms.

In [4]:
from openff.toolkit.utils.toolkits import GLOBAL_TOOLKIT_REGISTRY as tkr
from openff.toolkit.utils.toolkits import OpenEyeToolkitWrapper, RDKitToolkitWrapper, AmberToolsToolkitWrapper

# Deregister other toolkits to make sure we use OpenEye's
# implementation of AM1-BCC charging

tkr.deregister_toolkit(RDKitToolkitWrapper)
tkr.deregister_toolkit(AmberToolsToolkitWrapper)



In [ ]:
for i in range(len(molecules)):
    prefix = molecules[i].name.split(".")[0]
    print(prefix)
    sdf_file = prefix + "_charges.sdf"
    if not os.path.exists(sdf_file):
        molecules[i].assign_partial_charges(partial_charge_method="am1bcc")
        molecules[i].to_file(sdf_file, file_format='sdf')
    else:
        molecules[i] = Molecule.from_file(sdf_file)

pop_tetramer_mbuild
pop_hexamer_mbuild
pop_octamer_mbuild


/home/tfobe/Software/openff-toolkit/openff/toolkit/topology/molecule.py:2504: UserWarning: Warning! Partial charge method 'am1bcc' is not designed for use on large (i.e. > 150 atoms) molecules and may crash or take hours to run on this molecule (found 373 atoms). For more, see https://docs.openforcefield.org/projects/toolkit/en/stable/faq.html#parameterizing-my-system-which-contains-a-large-molecule-is-taking-forever-whats-wrong
  warnings.warn(


In [ ]:
tkr.register_toolkit(RDKitToolkitWrapper)
tkr.register_toolkit(AmberToolsToolkitWrapper)

## Writing Force Fields Topologies

First we load the speicfic force fields we want to write topologies with:

In [ ]:
force_fields = [
    ForceField("openff-2.0.0.offxml"),
    ForceField("openff-2.0.0_bespoke_pop_trimer.offxml")
]

ff_prefixes = ["off-2.0.0", "off-2.0.0-bespoke"]

In [ ]:
for ff,ff_prefix in zip(force_fields, ff_prefixes):
    for i in range(len(molecules)):
        prefix = molecules[i].name.split(".")[0]
        interchange = Interchange.from_smirnoff(
            force_field = ff,
            topology = openff_topologies[i],
            charge_from_molecules = [molecules[i]],
        )
        interchange.positions = pdbfiles[i].getPositions()
        
        # Convert to Gromacs format
        interchange.to_top(prefix + "_" + ff_prefix + ".top")
        interchange.to_gro(prefix + "_" + ff_prefix + ".gro")